In [1]:
PROJECT_ID = "gcp-mira-demo"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
#! gcloud auth login

In [3]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [4]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services import llm_generate
from services.query import query_prompts, embeddings
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource

from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                    DEFAULT_QUERY_EMBEDDING_MODEL)

In [5]:
query_engine = "lukman-test2"
doc_url = "gs://gcp-mira-demo-llm-docs"
user_id = "5nJrkPWa3D0yCKA853mD"
is_public=True

In [6]:
# create q_engine
llm_type = DEFAULT_QUERY_EMBEDDING_MODEL
q_engine = QueryEngine(name=query_engine,
                     created_by=user_id,
                     llm_type=llm_type,
                     is_public=is_public)
q_engine.save()

In [7]:
storage_client = storage.Client(project=PROJECT_ID)

vector_store = VectorStore(q_engine)

In [9]:
from services.query.query_service import process_documents

docs_processed, docs_not_processed = process_documents(doc_url, vector_store, q_engine, storage_client)

In [10]:
docs_processed

In [11]:
docs_not_processed

[]